<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/Longformer_for_sequence_classification_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!nvidia-smi

Mon Jun 15 16:05:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    32W / 250W |  13051MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!git clone https://github.com/huggingface/transformers.git
!pip install -U ./transformers
!pip install git+https://github.com/huggingface/nlp.git

Cloning into 'transformers'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 29311 (delta 11), reused 16 (delta 3), pack-reused 29278
Receiving objects: 100% (29311/29311), 26.63 MiB | 30.81 MiB/s, done.
Resolving deltas: 100% (20273/20273), done.
Processing ./transformers
     |████████████████████████████████| 3.8MB 4.5MB/s 
     |████████████████████████████████| 1.1MB 28.8MB/s 
     |████████████████████████████████| 890kB 23.6MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=688581 sha256=71571895398c11c76cd112f6112098e04a5f316a6f4bff2b407432f9314525ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-h9vu2ami/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ceffbe8a13b526cae5c5d60659c9787595cd72a5bab4c3704560bab6b7019781
  Sto

In [0]:
import torch
from transformers import LongformerTokenizerFast

In [2]:
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')


In [3]:
import nlp

dataset = nlp.load_dataset('imdb', 'plain_text')
print(dataset)

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0. Subsequent calls will reuse this data.
{'train': Dataset(schema: {'text': 'string', 'label': 'int64'}, num_rows: 25000), 'test': Dataset(schema: {'text': 'string', 'label': 'int64'}, num_rows: 25000), 'unsupervised': Dataset(schema: {'text': 'string', 'label': 'int64'}, num_rows: 50000)}


In [4]:
dataset["train"][10000]

{'label': 1,
 'text': "I really liked Tom Barman's AWTWB. You just have to let it come over you and enjoy it while it lasts, and don't expect anything. It's like sitting on a café-terrace with a beer, in the summer sun, and watching the people go by. It definitely won't keep you pondering afterwards, that's true, but that's not a prerequisite for a good film. It's just the experience during the movie that's great.<br /><br />I felt there were a few strands that could have been worked out a little more, but being a Lynch fan I don't care that much anymore :)<br /><br />And I *loved* the style, or flair of this movie. It's slick, but fresh, and the soundtrack is a beauty. Any music-lover will get his kicks out of AWTWB, I can assure you.<br /><br />I'll give it 8 out 10.<br /><br />(music-wise 10 out of 10)"}

In [0]:
# Tokenize our training dataset
def convert_to_features(example):
    # Tokenize contexts and questions (as pairs of inputs)
    encodings = tokenizer.encode_plus(example["text"], pad_to_max_length=True, max_length=512)    
    encodings.update({'label': example["label"]})

    return encodings



In [6]:
train_dataset  = nlp.load_dataset('imdb', 'plain_text', split=nlp.Split.TRAIN)
valid_dataset = nlp.load_dataset('imdb', 'plain_text', split=nlp.Split.TEST)


train_dataset = train_dataset.map(convert_to_features)
valid_dataset = valid_dataset.map(convert_to_features, load_from_cache_file=False)


# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'attention_mask','label']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)


25000it [00:20, 1241.45it/s]
25000it [00:19, 1258.82it/s]


In [7]:
len(train_dataset), len(valid_dataset)

(25000, 25000)

In [0]:
torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')


In [0]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch

from transformers import LongformerForSequenceClassification, LongformerTokenizerFast, EvalPrediction
from transformers import (
    HfArgumentParser,
    DataCollator,
    Trainer,
    TrainingArguments,
    set_seed,
)


logger = logging.getLogger(__name__)

# @dataclass
class DummyDataCollator(DataCollator):
    def collate_batch(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
            A dictionary of tensors
        """
        input_ids = torch.stack([example['input_ids'] for example in batch])
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        label = torch.stack([example['label'] for example in batch])

        return {
            'input_ids': input_ids, 
            'labels': label, 
            'attention_mask': attention_mask
        }


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """
    train_file_path: Optional[str] = field(
        default='train_data.pt',
        metadata={"help": "Path for cached train dataset"},
    )
    valid_file_path: Optional[str] = field(
        default='valid_data.pt',
        metadata={"help": "Path for cached valid dataset"},
    )
    max_len: Optional[int] = field(
        default=512,
        metadata={"help": "Max input length for the source text"},
    )


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

    # we will load the arguments from a json file, 
    # make sure you save the arguments in at ./args.json
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    tokenizer = LongformerTokenizerFast.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = LongformerForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )

    # Get datasets
    # print('loading data')
    # train_dataset  = torch.load(data_args.train_file_path)
    # valid_dataset = torch.load(data_args.valid_file_path)
    # print('loading done')

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=DummyDataCollator(),
        prediction_loss_only=True,
    )

    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    results = {}
    if training_args.do_eval and training_args.local_rank in [-1, 0]:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(eval_output.keys()):
                logger.info("  %s = %s", key, str(eval_output[key]))
                writer.write("%s = %s\n" % (key, str(eval_output[key])))
    
        results.update(eval_output)
    
    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()

In [0]:
import json
args_dict = {
  "n_gpu": 1,
  "model_name_or_path": 'allenai/longformer-base-4096',
  "max_len": 128 ,
  "output_dir": './models',
  "overwrite_output_dir": True,
  "per_gpu_train_batch_size": 8,
  "per_gpu_eval_batch_size": 8,
  "gradient_accumulation_steps": 16,
  "learning_rate": 1e-4,
  "num_train_epochs": 3,
  "do_train": True,
  "do_eval" : True
}


with open('args.json', 'w') as f:
  json.dump(args_dict, f)



In [11]:
main()

06/15/2020 14:16:39 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 14:16:40 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 14:16:41 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 14:16:42 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 14:16:43 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 14:16:44 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 14:16:45 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 14:16:46 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 14:16:47 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS t

06/15/2020 14:17:20 - INFO - transformers.modeling_utils -   Model weights saved in ./models/pytorch_model.bin


{}

In [12]:
tokenizer = LongformerTokenizerFast.from_pretrained(
    "allenai/longformer-base-4096"
)
model = LongformerForSequenceClassification.from_pretrained(
    "./models"
)

06/15/2020 14:32:48 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json from cache at /root/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
06/15/2020 14:32:48 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-merges.txt from cache at /root/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
06/15/2020 14:32:49 - INFO - transformers.configuration_utils -   loading configuration file ./models/config.json
06/15/2020 14:32:49 - INFO - transformers.configuration_utils -   Model config LongformerConfig {
  "architectures": [
    "LongformerForSequenceClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropo

In [40]:
model.cuda()
model.eval()

def simple_accuracy(preds, labels):
  return (preds == labels).mean() 

label_ids: torch.Tensor = None
preds: torch.Tensor = None

from tqdm.auto import tqdm

with torch.no_grad():
  dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, collate_fn=DummyDataCollator().collate_batch)


  for batch in tqdm(dataloader):
    predictions = model(input_ids=batch['input_ids'].cuda(),
                                  attention_mask=batch['attention_mask'].cuda())
    
    predictions = predictions[0]

    if preds is None:
      preds = predictions.detach()
    else:
      preds = torch.cat((preds, predictions.detach()), dim=0)


    if label_ids is None:
      label_ids = batch["labels"].detach()
    else:
      label_ids = torch.cat((label_ids, batch["labels"].detach()), dim=0)
    


preds = np.argmax(preds.cpu().numpy(), axis=1)
label_ids = label_ids.cpu().numpy()

print(simple_accuracy(preds, label_ids))

06/15/2020 15:36:08 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 15:36:08 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 15:36:09 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 15:36:09 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 15:36:10 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 15:36:10 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 15:36:11 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 15:36:11 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS token...
06/15/2020 15:36:11 - INFO - transformers.modeling_longformer -   Initializing global attention on CLS t


0.94996


In [0]:
VERSION = "nightly"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

In [14]:
!pip install tf-nightly

     |████████████████████████████████| 350.2MB 47kB/s 
     |████████████████████████████████| 460kB 40.2MB/s 
     |████████████████████████████████| 3.0MB 43.0MB/s 


In [1]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.82.205.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.82.205.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


In [17]:
import torch_xla.distributed.xla_multiprocessing as xmp
xmp.spawn(_mp_fn, args=(), nprocs=8, start_method='fork')

Exception in device=TPU:0: Error while lowering: f32[96,2,768,64]{3,2,1,0} aten::as_strided, size=(96, 2, 768, 64), stride=(65536, 16384, 64, 1), storage_offset=0
Error: torch_xla/csrc/ops/as_strided.cpp:25 : Check failed: storage_offset + slice_size <= input_element_count (9437184 vs. 6291456)
*** Begin stack trace ***
	tensorflow::CurrentStackTrace[abi:cxx11]()
	torch_xla::ir::ops::AsStrided::Lower(torch_xla::ir::LoweringContext*) const
	torch_xla::ir::LoweringContext::LowerNode(torch_xla::ir::Node const*)
	torch_xla::ir::LoweringContext::LoweringContext(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, torch_xla::Device, absl::lts_2020_02_25::Span<torch_xla::ir::Node const* const>, std::unordered_map<torch_xla::ir::Node const*, torch_xla::ir::Util::EmitStatus, std::hash<torch_xla::ir::Node const*>, std::equal_to<torch_xla::ir::Node const*>, std::allocator<std::pair<torch_xla::ir::Node const* const, torch_xla::ir::Util::EmitStatus> > >)
	torch_xl

Exception: ignored